In [180]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np

In [181]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open("./preferenceGraph.json") as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], comment=edge["comment"])
    
    return G

In [182]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    degrees = [0 for i in range(n)]


    def calculateProbability(weight) :
        if re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
            return 0.7
        elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
            return 0.3
        elif re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
            return 1
        else :
            return 0
        
    def analyzeQuiz() :        
        #dictionary {edge : probability of existing in the Preference Graph}
        edgesProbabilities = {}

        #examine each eage in Preference Graph
        #calculate probability. How strong is the connection between people?
        for u, v in preferenceGraph.edges() :
            if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
                if (v,u) in edgesProbabilities :
                    #(v,u) is already checked, so neglect (u,v)
                    continue 
                else :
                    #(u,v), (v,u) pair has not been checked
                    edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["comment"]) + calculateProbability(preferenceGraph[v][u]["comment"]))
            else :
                edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["comment"])

        #find possible peers for a student
        #step 1: find connecting links with prob < threshold
        possiblePeers = [[] for _ in range(n)]       
        for edge, prob in edgesProbabilities.items() :
            if prob < threshold :
                possiblePeers[edge[0]].append(edge[1])
                possiblePeers[edge[1]].append(edge[0])

        #step 2: keep all the other nodes as possible peers to interact with
        possiblePeers = list(map(lambda item : list(set(range(n)) - set(item[1] + [item[0]])), enumerate(possiblePeers)))
        
        return possiblePeers
    
    

    def addEdgesAtTimestamp() :
        numberOfInteractions = np.random.choice([2,3,4])
        G = nx.Graph()
        G.add_nodes_from(range(n))

        for node in range(n) :
            #find for a node how many additional interactions could we add with respect to maxInteractions parameter
            possibleInteractions = maxInteractions - degrees[node]
            if possibleInteractions < 0 :
                continue
            
            #choose randomly from possible peers
            for interaction in np.random.choice(possiblePeers[node], size=min(numberOfInteractions, possibleInteractions), replace=False) :
                G.add_edge(node, interaction)
        
        interactionGraph.append(G)


    possiblePeers = analyzeQuiz()

    #interactions at timetsamp t = 0
    #interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3))
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    #interactions at timestamps t >= 1
    for time in range(1, maxTimestamp+1) :
        addEdgesAtTimestamp()

        for u, v in interactionGraph[time].edges() :
            interactionGraph[time][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))

        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 
        if nx.is_empty(interactionGraph[time]) :
            interactionGraph += [nx.Graph() for _ in range(maxTimestamp - time)]
            break

    
    return interactionGraph

In [183]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 24,
    "maxTimestamp" : 5
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

In [184]:
print("Snapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G[i]}")

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 294 edges
at t = 2: Graph with 100 nodes and 282 edges
at t = 3: Graph with 100 nodes and 340 edges
at t = 4: Graph with 100 nodes and 114 edges
at t = 5: Graph with 100 nodes and 17 edges


In [185]:
activities = ["to share content, chat in a social network platform",
            "to participate in debate/group presentation & discussion",
            "to play games",
            "to study in group or work on collaborative projects",
            "to be volunteers",
            "to be teammates in a sports team"]

for i,j,k in G[0].edges(data=True) :
    print(f"({i},{j}) => interaction: {activities[k['interaction_index']]}")

(0,1) => interaction: to be volunteers
(0,2) => interaction: to play games
(0,3) => interaction: to be teammates in a sports team
(0,4) => interaction: to play games
(0,5) => interaction: to be teammates in a sports team
(0,6) => interaction: to play games
(0,7) => interaction: to play games
(0,8) => interaction: to be volunteers
(0,9) => interaction: to play games
(0,11) => interaction: to participate in debate/group presentation & discussion
(0,12) => interaction: to study in group or work on collaborative projects
(0,14) => interaction: to study in group or work on collaborative projects
(0,16) => interaction: to play games
(0,23) => interaction: to be teammates in a sports team
(0,28) => interaction: to participate in debate/group presentation & discussion
(0,36) => interaction: to study in group or work on collaborative projects
(0,43) => interaction: to be teammates in a sports team
(0,53) => interaction: to be teammates in a sports team
(0,56) => interaction: to study in group o